# Алгоритм получения хэша фармакофора

Получаем фармакофор в виде таблицы координат центров 

$$
A =
\left[ \begin{array}
xx_1 & y_1 & z_1 \\
x_2 & y_2 & z_2 \\
x_3 & y_3 & z_3 \\
x_4 & y_4 & z_4 \\
\end{array} \right]
$$

Напишем функцию, которая возвращает центр массы фигуры. Каждому типу вершин присвоим какое-то иррациональное число (корень из простого числа) в качестве веса. <br />
Далее координаты каждой вершины $\vec{X}_i = [x_i, y_i, z_i]$ умножим на ее вес $w_i$. Затем найдем сумму из координат и разделим это все на сумму всех весов:

$$
\vec{c} = \frac{\sum_{i=1}^4 \vec{X}_i w_i}{\sum_{i=1}^4 w_i}
$$

Функция возвращает вектор $\vec{c} = [x_c, y_c, z_c]$

Напишем функцию, перемещающую фигуру так, чтобы центр масс находился в начале координат. Для этого из каждого вектора вершины вычтем вектор центра масс. 

$$
\vec{X}_{i0} = \vec{X}_i - \vec{c }
$$


Напишем функцию, которая возвращает тензор инерции фигуры. 

$$
I = \left[ \begin{array}
II_{xx} & I_{xy} & I_{xz} \\
I_{yx} & I_{yy} & I_{yz} \\
I_{zx} & I_{zy} & I_{zz} \\
\end{array} \right]
$$

Компоненты тензора инерции вычисляются по формулам:

$$
I_{xx} = \sum_{i=1}^4 w_i (z_i^2 + y_i^2); \hspace{2cm}
I_{yy} = \sum_{i=1}^4 w_i (x_i^2 + z_i^2); \hspace{2cm}
I_{zz} = \sum_{i=1}^4 w_i (x_i^2 + y_i^2); \\
I_{xy} = I_{xy} = - \sum_{i=1}^4 w_i x_i y_i \\
I_{xz} = I_{zx} = - \sum_{i=1}^4 w_i x_i z_i \\
I_{yz} = I_{zy} = - \sum_{i=1}^4 w_i z_i y_i \\
$$

Найдем собственные векторы $\vec{e}$  и собственные значения $\lambda$ тензора моментов инерции. 

$$
I \vec{e} = \lambda \vec{e} 
$$

Получаем три собственных значения и три соответствующих им собственных вектора. Поскольку матрица симметрична относительно главной оси - все значения $\lambda$ будут вещественными.

Запишем их в переменные 'w' $ =[ \lambda_1, \lambda_2, \lambda_3 ] $ и 'v' = $ R = [\vec{e}_1 \vec{e}_2 \vec{e}_3] $
Затем отсортируем собственные значения и соответствующие им векторы, так чтобы $ \lambda_x \geqslant \lambda_y \geqslant \lambda_z $, то есть наиболее "тяжелое" направление будет проходить вдоль оси $x$, наименее "тяжелое" - вдоль $z$. Матрица собственных векторов будет выглядеть как $R = [\vec{e}_x \vec{e}_y \vec{e}_z]$. Эти векторы образуют ортонормированный базис. 

Для того, чтобы перебрать все возможные варианты матриц $R$, нам нужно домножить составляющие их векторы либо на +1, либо на -1. Для этого нужно получить все возможные размещения с повторениями чисел 1 и -1 по 3.

Итак, для каждого $\lambda_i$ существует два вектора $\vec{e}_i$ и $-\vec{e}_i$, направленных в разные стороны, значит у нас есть 8 различных варинатов матрицы $R$. 

$$ R_i = R \left[ \begin{array}
0\pm 1 & 0 & 0 \\
0 & \pm1 & 0 \\
0 & 0 & \pm 1 \\
\end{array} \right]
$$

Надо перебрать их все. Половина из них будет правой тройкой векторов, вторая половина - левой. Отбрасываем все левые тройки векторов, так как при умножении на них мы, помимо поворота, зеркально отразим фигуру, что нежелательно. Для того, чтобы определить, правая это тройка или левая нужно найти объем параллелепипеда, натянутого на эти векторы.

$$
V = \vec{e}_x \cdot [\vec{e}_y \times \vec{e}_z]
$$

Если объем положительный - это правая тройка, если объем отрицательный - это левая тройка.
В итоге мы получим набор матриц $R_1, R_2, R_3, R_4$.


Найдем новые координаты вершин. Координаты $i$-го центра:
$$
\vec{x}_{i1} = \vec{x}_{i0} R_i
$$

Новые координаты можем получить, умножив справа матрицу координат $A_0$ на матрицу $R_i$:

$$
A_{i1} = A_0 R_i
$$

После чего округляем результат до заданного интервала.


Для этого напишем функцию округления, которая принимает два числа $n$ - число, которое нужно округлить,
и $m$ - точность округления. 
Функция возвращает $n - n \cdot mod(m)$, если $n \cdot mod(m) > m/2$, иначе: $n - n \cdot mod(m) + m$.
Таким образом, мы можем округлять не только до целых, десятых и т.д., но и до любого заданного положительного значения: 2, 3, 1.5, 2/3 и т.д.

Этой функцией проходим по всем значениям матрицы, получая матрицу округленных значений.

В результате имеем четыре матрицы $A_{i1}$. Отсортируем кортежи с вершинами и координатами по весу вершины, начиная с самой тяжелой.

Затем отсортируем эти матрицы по значениям координат и выберем первый из них. Это и будет искомый хэш фармакофора.